1. Установить последнюю версию docker из официальных apt репозиториев.

2. Запустите контейнер с именем web со следующими параметрами:
- Имя контейнера должно быть `web`
- Образ контейнера должен быть `nginx:latest` (с обязательным указанием тега)
- Контейнер должен быть доступен на порту `9099`
- Директория `/var/log/nginx` внутри контейнера должна быть смонтирована в директорию `/data/nginx/logs` на хост системе
- При запуске контейнера должна быть добавлена переменная окружения `logging` = `true`

In [ ]:
docker run -d --name web -p 9099:80 -v /data/nginx/logs:/var/log/nginx -e logging=true nginx:latest

1. Установить последнюю версию docker из официальных apt репозиториев.

2. Настроить docker daemon таким образом, чтобы он слушал входящие подключения на всех интерфейсах (0.0.0.0) на порту 2375 без использования TLS.

3. Создать контекст с именем remote-docker, который будет подключаться к вашему docker daemon по любому интерфейсу, кроме loopback интерфейса.

4. Создайте две сети с драйвером bridge и именами cache и front.

5. Запустите контейнер с именем redis и образом redis:latest (с обязательным указанием тега) в сети cache.

6. Подключите контейнер redis к сети front.

In [ ]:
sudo nano /etc/docker/daemon.json
{
  "hosts": ["tcp://0.0.0.0:2375", "unix:///var/run/docker.sock"]
}
docker context create remote-docker tcp://0.0.0.0:2375
docker network create --driver=bridge cache
docker network create --driver=bridge front
docker run --name redis --network=cache redis:latest
docker network connect front redis

1. Установить последнюю версию docker из официальных apt репозиториев.

2. Создайте директорию /data/logs на хост системе с правами 777 (всем доступно все - chmod 777 /data/logs)

3. Запустите контейнер с именем fluentd со следующими условиями:
- Контейнер должен пробрасывать порт 24224 внутрь контейнера
- Директория /fluentd/log внутри контейнера должна быть подмонтирована к /data/logs на хост системе
- Контейнер должен использовать образ fluentd:latest (с обязательным указанием тега)

4. Запустите контейнер с именем nginx со следующими условиями:
- Контейнер должен использовать образ nginx:latest (с обязательным указанием тега)
- Драйвер логирования должен быть указан fluentd
- Адрес fluentd сервера должен быть - 127.0.0.1:24224.

5. Проверьте наличие логов в директории /data/logs.

In [ ]:
sudo mkdir -p /data/logs
sudo chmod 777 /data/logs
sudo docker run -d --name fluentd -p 24224:24224 -v /data/logs:/fluentd/log fluentd:latest
sudo docker run -d --name nginx --log-driver fluentd --log-opt fluentd-address=127.0.0.1:24224 nginx:latest


1. Установить последнюю версию docker из официальных apt репозиториев.

2. Создайте именованный volume с именем html

3. Запустите контейнер с именем data в интерактивном режиме со следующими параметрами:
- Контейнер должен монтировать именованный volume 'html' в /data внутри контейнера
- Контейнер должен использовать образ alpine:latest (с обязательным указанием тега)

4. Внутри контейнера запишите внутрь файла /data/index.html строку "INDEX PAGE"

5. Выйдите из контейнера data

6. Запустите контейнер с именем nginx со следующими условиями:
- Контейнер должен использовать образ nginx:latest (с обязательным указанием тега)
- Контейнер должен монтировать volume 'html' в /usr/share/nginx/html в режиме только для чтения
- Порт 9099 должен быть проброшен в контейнер на порт 80

7. Попробуйте запросить страницу 127.0.0.1:9099 с помощью curl и проверьте ответ.

In [ ]:

docker volume create html
docker run -dit --name data -v html:/data alpine:latest
echo "INDEX PAGE" > /data/index.html
exit
docker run -dit --name nginx -v html:/usr/share/nginx/html:ro -p 9099:80 nginx:latest
curl 127.0.0.1:9099


1. Установить последнюю версию docker из официальных apt репозиториев.

2. Создайте директорию /data/logs на хост системе с правами 777 (всем доступно все - chmod 777 /data/logs)

3. Запустите контейнер с именем fluentd со следующими условиями:
- Контейнер должен пробрасывать порт 24224 внутрь контейнера
- Директория /fluentd/log внутри контейнера должна быть подмонтирована к /data/logs на хост системе
- Контейнер должен использовать образ fluentd:latest

4. Создайте сеть `back`

5. Создайте volume с именем `mysql`

6. Запустите контейнер с базой и именем mysql:

    Контейнер должен использовать образ mysql:5.7

    В контейнер должна передаваться ENV переменная MYSQL_DATABASE=wpdb

    В контейнер должна передаваться ENV переменная MYSQL_USER=wpuser

    В контейнер должна передаваться ENV переменная MYSQL_PASSWORD=wppass

    В контейнер должна передаваться ENV переменная MYSQL_RANDOM_ROOT_PASSWORD=1

    Volume mysql должен монтироваться в директорию /var/lib/mysql внутри контейнера

    Контейнер должен находиться в сети back

    Контейнер должен использовать драйвер логирования fluentd и отправлять логи на 127.0.0.1:24224

7. Создайте volume с именем `uploads`

8. Запустите контейнер с именем wordpress:

    Контейнер должен использовать образ wordpress:latest

    В контейнер должна передаваться ENV переменная WORDPRESS_DB_HOST=mysql

    В контейнер должна передаваться ENV переменная WORDPRESS_DB_USER=wpuser

    В контейнер должна передаваться ENV переменная WORDPRESS_DB_PASSWORD=wppass

    В контейнер должна передаваться ENV переменная WORDPRESS_DB_NAME=wpdb

    Volume uploads должен монтироваться в директорию /var/www/html внутри контейнера 

    Контейнер должен находиться в сети back

    Внешний порт 80 должен пробрасываться на порт 80 внутри контейнера

    Контейнер должен использовать драйвер логирования fluentd и отправлять логи на 127.0.0.1:24224

9. Введите в строке браузера IP адрес вашего сервера и убедитесь, что wordpress доступен.

In [ ]:
mkdir /data/logs
chmod 777 /data/logs


docker network create back
docker volume create mysql
docker volume create uploads

docker run --rm -d -p 24224:24224 -v /data/logs:/fluentd/log --name fluentd fluentd:latest 


docker run -d --name mysql \
--rm\
-e MYSQL_DATABASE="wpdb" \
-e MYSQL_USER="wpuser" \
-e MYSQL_PASSWORD="wppass" \
-e MYSQL_RANDOM_ROOT_PASSWORD=1 \
-v mysql:/var/lib/mysql \
--network=back \
--log-driver=fluentd \
--log-opt fluentd-address=127.0.0.1:24224 \
mysql:5.7

docker run \
--rm \
--name wordpress \
-d \
-e WORDPRESS_DB_HOST="mysql" \
-e WORDPRESS_DB_USER="wpuser" \
-e WORDPRESS_DB_PASSWORD="wppass" \
-e WORDPRESS_DB_NAME="wpdb" \
-v uploads:/var/www/html \
--network=back \
-p 80:80 \
--log-driver=fluentd \
--log-opt fluentd-address=127.0.0.1:24224 \
wordpress:latest 